# Train SVM classifiers for poet profiling

In [1]:
from format import *
from features import *
from svm_classifier import *
import torch
import torch.optim
import torchtext

/Users/menga/opt/anaconda3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load in the training data. Then extract features to turn poem into vector.

In [2]:
df_train, df_test = get_text_to_gender()

In [3]:
# what does F vs M train split look like
df_train.where(df_train['gender']=='F').count()

content    275
gender     275
dtype: int64

Some features (BoW and prosodic attributes)

In [4]:
# unigram and bigrams
unigrams, bigrams = get_top_n_vocab(df_train['content'], n=250)
print(unigrams)
print(bigrams)

['de', 'y', 'la', 'el', 'que', 'en', 'a', 'su', 'del', 'al', 'tu', 'mi', 'con', 'un', 'los', 'se', 'las', 'no', 'por', 'es', 'me', 'más', 'sus', 'te', 'una', 'como', 'lo', 'si', 'amor', 'sin', 'Y', 'ni', 'vida', 'tus', 'le', 'alma', 'para', 'ya', 'mis', 'tan', 'cuando', 'o', 'luz', 'cielo', 'yo', 'pues', 'Dios', 'sol', 'ser', 'ha', 'entre', 'mundo', 'No', 'corazón', 'En', 'ojos', 'sobre', 'bien', 'La', 'El', 'qué', 'gloria', 'muerte', 'todo', 'tú', 'día', 'quien', 'cual', 'porque', 'ti', 'dulce', 'son', 'mar', 'De', 'donde', 'dolor', 'sólo', 'Si', 'noche', 'siempre', 'fue', 'tanto', 'triste', 'Mas', 'frente', 'fin', 'flores', 'voz', 'tiempo', 'tierra', 'mano', 'pecho', 'este', 'Oh', 'mas', 'hay', 'está', 'oh', 'tal', 'así', 'nos', 'oro', 'mí', 'hoy', 'Por', 'mal', 'hasta', 'mía', 'Qué', 'esta', 'vez', 'él', 'viento', 'Yo', 'dos', 'ella', 'e', 'aunque', 'he', 'fuego', 'llanto', 'llama', 'bajo', 'flor', 'aquel', 'pero', 'A', 'Es', 'Que', 'hombre', 'vano', 'gran', 'suerte', 'sombra', 'Ya'

In [5]:
# using the scansion is apparently horridly slow, so parallelize?
# HONESTLY, still pretty slow so just don't use this code and work only with n-grams
# out = df_train['content'].head(5).apply(get_metrical_vector)
# print(out)

In [6]:
# compute ngram and prosodic features
# N x D
input = []
counter = 0
total = df_train.count()['content']
for text in df_train['content']:
    bow = text_to_bag_of_words(text, unigrams, bigrams)
    input.append(bow)
    #prosodic = get_metrical_vector(text)
    #input.append(list(bow) + list(prosodic))
    if counter % 1000 == 0:
        print(f"Constructing feature vector {counter}/{total}")
    counter += 1

input_arr = torch.Tensor(input).reshape((total, -1))
df_train['gender'] = df_train['gender'].map({'M': 0, 'F': 1})


label_arr = torch.Tensor(df_train['gender'].values).reshape((-1, 1))


Constructing feature vector 0/4045
Constructing feature vector 1000/4045
Constructing feature vector 2000/4045
Constructing feature vector 3000/4045
Constructing feature vector 4000/4045


/var/folders/mg/3jf2y33s1gl75f_3gb1l8sbh0000gn/T/ipykernel_44713/3158854010.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  input_arr = torch.Tensor(input).reshape((total, -1))


Create SVM model.

In [16]:

model = MulticlassSVM(input_arr.shape[1], 2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-3)


epochs = 5

for epoch in range(epochs):
    outputs = model(input_arr)

    correct_class_scores = outputs[range(2), label_arr.int()]

    print(correct_class_scores)
        
    # Compute the margin-based loss for all classes except the correct class
    print(outputs.shape)
    print(correct_class_scores.shape)
    diff = outputs - correct_class_scores
    margins = torch.max(diff + 1.0, 0, keepdim=True)


    #### WHAT'S HAPPENING
    loss = model.hinge_loss(outputs, label_arr.int())
    print(f'Epoch {epoch}: Hinge Loss is {loss.item()}')

    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

tensor([[-0.0240, -0.0835],
        [-0.0240, -0.0835],
        [-0.0240, -0.0835],
        ...,
        [-0.0240, -0.0835],
        [-0.2448,  0.0877],
        [-0.0240, -0.0835]], grad_fn=<IndexBackward0>)
torch.Size([4045, 2])
torch.Size([4045, 2])
torch.Size([4045, 2])
torch.Size([4045, 1])


RuntimeError: The size of tensor a (2) must match the size of tensor b (4045) at non-singleton dimension 1